In [1]:
# 📦 Step 0: Import Required Libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.version.cuda)

True
1
12.1


In [ ]:
# Add the parent directory to the Python path
# to handle ModuleNotFoundError: No module named 'models' error
import sys
sys.path.append('../') 

from models.lstm_forecast_model import LSTMForecast
from models.timeseries_dataset_class import TimeSeriesDataset


# ✅ Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

In [ ]:
# Parameters
window_size = 48
batch_size = 32
epochs = 10
learning_rate = 1e-3
target_column = "OT"

In [ ]:
# 📂 Step 1: Load split datasets
train_df = pd.read_csv("../data/processed/etth1_train.csv", parse_dates=["date"], index_col="date")
val_df = pd.read_csv("../data/processed/etth1_val.csv", parse_dates=["date"], index_col="date")
test_df = pd.read_csv("../data/processed/etth1_test.csv", parse_dates=["date"], index_col="date")

train_target = train_df[target_column].values
val_target = val_df[target_column].values
test_target = test_df[target_column].values

In [ ]:

# 📊 Step 2: Prepare TimeSeriesDataset and DataLoaders

train_loader = DataLoader(TimeSeriesDataset(train_target, window_size), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(val_target, window_size), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TimeSeriesDataset(test_target, window_size), batch_size=batch_size, shuffle=False)


In [ ]:
# ⚙️ Step 3: Define and Train the Model
model = LSTMForecast().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device).unsqueeze(1)
        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}] Train Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
# 💾 Step 4: Save the trained model
os.makedirs("../models/checkpoints", exist_ok=True)
torch.save(model.state_dict(), "../models/checkpoints/lstm_model.pth")
print("✅ Model saved to checkpoints/lstm_model.pth")

In [ ]:
# SMAPE calculation
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0  # Avoid division by zero
    return 100 * np.mean(diff)


In [ ]:
# 📈 Step 5: Validate and Plot Predictions

def evaluate_model(dataloader, title, plot_name):
    model.eval()
    all_preds = []
    all_trues = []
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            pred = model(x).cpu().numpy()
            all_preds.append(pred)
            all_trues.append(y.numpy())


    predictions = np.concatenate(all_preds)
    truths = np.concatenate(all_trues)

    plt.figure(figsize=(10, 5))
    plt.plot(truths[:100], label="True")
    plt.plot(predictions[:100], label="Predicted")
    plt.legend()
    plt.title(title)
    plt.grid(True)
    
    os.makedirs("../../outputs", exist_ok=True)
    plt.savefig(f"../../outputs/{plot_name}")
    print(f"✅ Plot saved to outputs/{plot_name}")
    plt.show()


    # 📊 Metrics
    mse = mean_squared_error(truths, predictions)
    mae = mean_absolute_error(truths, predictions)
    smape_val = smape(truths, predictions)

    print(f"📊 {title} Metrics:")
    print(f" - MSE   : {mse:.6f}")
    print(f" - MAE   : {mae:.6f}")
    print(f" - SMAPE : {smape_val:.2f}%")
    
    return predictions, truths

In [ ]:
# Evaluate on validation and test sets
evaluate_model(val_loader, "📈 Validation Set Prediction vs Ground Truth", "lstm_model_val_predictions.png")
evaluate_model(test_loader, "📈 Test Set Prediction vs Ground Truth", "lstm_model_test_predictions.png")